In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


# 1. Data Pre-Processing

In [2]:
df_ROI_for_ML_Opti = pd.read_csv(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_FNA/df_ROIuMAP1_for_ML_Opti.csv").set_index('pixel_id')
df_ROI_for_ML_Opti_train = pd.read_csv(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_FNA/df_ROIuMAP1_for_ML_Opti_train.csv").set_index('pixel_id')
df_ROI_for_ML_Opti_ext = pd.read_csv(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_FNA/df_ROIuMAP1_for_ML_Opti_ext.csv").set_index('pixel_id')
df_ROI_for_ML_Opti_ingested = pd.read_csv(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_FNA/df_ROIuMAP1_for_ML_Opti_ingested.csv").set_index('pixel_id')
df_ROI_for_ML_Opti_FNA = pd.read_csv(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_FNA/df_ROIuMAP1_for_ML_Opti_FNA.csv").set_index('pixel_id')
df_ROI_for_ML_Opti_DirectIn = pd.read_csv(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_FNA/df_ROIuMAP1_for_ML_Opti_DirectIn.csv").set_index('pixel_id')

In [3]:
df_ROI_for_ML_Opti_DirectIn

,106.9792,108.0186,117.0458,124.0068,124.9906,125.0094,126.0017,126.9036,135.0298,151.0214,...,883.5338,885.5488,886.5521,887.5599,896.7424,919.7392,929.5816,1058.7522,1448.9673,type
pixel_id,,,,,,,,,,,,,,,,,,,,,
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_2_1,31467.708984,0.000000,0.0,8226763.0,0.000000,68493.031250,209007.734375,157402.703125,159659.109375,0.000000,...,235876.031250,1331585.625,6.372926e+05,216978.078125,0.0,0.000000e+00,0.000000e+00,0.00000,0.0,0
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_4_1,71850.296875,54996.867188,0.0,10210607.0,76446.265625,75615.703125,324880.343750,176383.625000,294884.968750,201689.750000,...,196964.875000,1451490.000,5.922894e+05,350863.875000,0.0,9.053408e+05,0.000000e+00,0.00000,0.0,0
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_5_1,45051.843750,158222.312500,0.0,11721988.0,46072.605469,244047.828125,403619.437500,0.000000,346991.656250,367011.718750,...,319688.312500,1853836.250,9.405958e+05,273320.562500,0.0,1.346318e+06,0.000000e+00,194159.53125,0.0,0
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_9_1,126403.914062,137242.171875,0.0,11873774.0,0.000000,185851.468750,365448.375000,216357.812500,333092.656250,281232.656250,...,345452.562500,1594346.000,7.149868e+05,310046.593750,0.0,0.000000e+00,0.000000e+00,0.00000,0.0,0
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_11_1,76320.437500,89533.828125,0.0,10862974.0,64757.335938,167189.359375,356060.531250,84716.257812,341082.531250,0.000000,...,246124.218750,1256030.875,6.319585e+05,204829.156250,0.0,1.192396e+06,0.000000e+00,0.00000,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_594_1,208032.921875,93289.890625,0.0,14243413.0,191679.906250,260365.359375,417522.500000,183456.328125,342139.187500,0.000000,...,340221.156250,1470005.250,7.135132e+05,518926.687500,4410163.5,0.000000e+00,1.283354e+06,0.00000,0.0,1
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_595_1,43703.371094,0.000000,0.0,7453880.0,71115.992188,135152.984375,280401.156250,78532.375000,142307.203125,111165.835938,...,294071.910156,2130823.750,1.099195e+06,437886.093750,0.0,0.000000e+00,6.556245e+04,0.00000,0.0,1
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_596_1,334999.531250,226093.421875,0.0,19376702.0,179186.796875,309036.312500,637430.312500,77071.039062,442702.937500,277333.437500,...,176609.109375,1115554.000,5.783931e+05,258386.812500,0.0,0.000000e+00,0.000000e+00,0.00000,7080241.0,1


In [4]:
df_ROI = df_ROI_for_ML_Opti.reset_index()
df_ROI_train = df_ROI_for_ML_Opti_train.reset_index()
df_ROI_ext = df_ROI_for_ML_Opti_ext.reset_index()
df_ROI_ingested = df_ROI_for_ML_Opti_ingested.reset_index()
df_ROI_FNA = df_ROI_for_ML_Opti_FNA.reset_index()
df_ROI_DirectIn = df_ROI_for_ML_Opti_DirectIn.reset_index()

In [5]:
df_ROI["Sample"] = 0
df_ROI["Row"] = 0
df_ROI["Scan"] = 0
df_ROI_Sample = []
df_ROI_Row = []
df_ROI_Scan = []
for i in range(len(df_ROI["pixel_id"])):
    df_ROI_Sample.append("_".join(str(df_ROI["pixel_id"][i]).split("_")[:-2]))
    df_ROI_Row.append(int(str(df_ROI["pixel_id"][i]).split("_")[-1]))
    df_ROI_Scan.append(int(str(df_ROI["pixel_id"][i]).split("_")[-2]))
df_ROI["Sample"] = df_ROI_Sample
df_ROI["Row"] = df_ROI_Row
df_ROI["Scan"] = df_ROI_Scan
pop_column = df_ROI.pop('Scan')
df_ROI.insert(1, 'Scan', pop_column)
pop_column = df_ROI.pop('Row')
df_ROI.insert(1, 'Row', pop_column)
pop_column = df_ROI.pop('Sample')
df_ROI.insert(1, 'Sample', pop_column)

In [6]:
df_ROI_train["Sample"] = 0
df_ROI_train["Row"] = 0
df_ROI_train["Scan"] = 0
df_ROI_train_Sample = []
df_ROI_train_Row = []
df_ROI_train_Scan = []
for i in range(len(df_ROI_train["pixel_id"])):
    df_ROI_train_Sample.append("_".join(str(df_ROI_train["pixel_id"][i]).split("_")[:-2]))
    df_ROI_train_Row.append(int(str(df_ROI_train["pixel_id"][i]).split("_")[-1]))
    df_ROI_train_Scan.append(int(str(df_ROI_train["pixel_id"][i]).split("_")[-2]))
df_ROI_train["Sample"] = df_ROI_train_Sample
df_ROI_train["Row"] = df_ROI_train_Row
df_ROI_train["Scan"] = df_ROI_train_Scan
pop_column = df_ROI_train.pop('Scan')
df_ROI_train.insert(1, 'Scan', pop_column)
pop_column = df_ROI_train.pop('Row')
df_ROI_train.insert(1, 'Row', pop_column)
pop_column = df_ROI_train.pop('Sample')
df_ROI_train.insert(1, 'Sample', pop_column)

df_ROI_ext["Sample"] = 0
df_ROI_ext["Row"] = 0
df_ROI_ext["Scan"] = 0
df_ROI_ext_Sample = []
df_ROI_ext_Row = []
df_ROI_ext_Scan = []
for i in range(len(df_ROI_ext["pixel_id"])):
    df_ROI_ext_Sample.append("_".join(str(df_ROI_ext["pixel_id"][i]).split("_")[:-2]))
    df_ROI_ext_Row.append(int(str(df_ROI_ext["pixel_id"][i]).split("_")[-1]))
    df_ROI_ext_Scan.append(int(str(df_ROI_ext["pixel_id"][i]).split("_")[-2]))
df_ROI_ext["Sample"] = df_ROI_ext_Sample
df_ROI_ext["Row"] = df_ROI_ext_Row
df_ROI_ext["Scan"] = df_ROI_ext_Scan
pop_column = df_ROI_ext.pop('Scan')
df_ROI_ext.insert(1, 'Scan', pop_column)
pop_column = df_ROI_ext.pop('Row')
df_ROI_ext.insert(1, 'Row', pop_column)
pop_column = df_ROI_ext.pop('Sample')
df_ROI_ext.insert(1, 'Sample', pop_column)

df_ROI_ingested["Sample"] = 0
df_ROI_ingested["Row"] = 0
df_ROI_ingested["Scan"] = 0
df_ROI_ingested_Sample = []
df_ROI_ingested_Row = []
df_ROI_ingested_Scan = []
for i in range(len(df_ROI_ingested["pixel_id"])):
    df_ROI_ingested_Sample.append("_".join(str(df_ROI_ingested["pixel_id"][i]).split("_")[:-2]))
    df_ROI_ingested_Row.append(int(str(df_ROI_ingested["pixel_id"][i]).split("_")[-1]))
    df_ROI_ingested_Scan.append(int(str(df_ROI_ingested["pixel_id"][i]).split("_")[-2]))
df_ROI_ingested["Sample"] = df_ROI_ingested_Sample
df_ROI_ingested["Row"] = df_ROI_ingested_Row
df_ROI_ingested["Scan"] = df_ROI_ingested_Scan
pop_column = df_ROI_ingested.pop('Scan')
df_ROI_ingested.insert(1, 'Scan', pop_column)
pop_column = df_ROI_ingested.pop('Row')
df_ROI_ingested.insert(1, 'Row', pop_column)
pop_column = df_ROI_ingested.pop('Sample')
df_ROI_ingested.insert(1, 'Sample', pop_column)

df_ROI_FNA["Sample"] = 0
df_ROI_FNA["Row"] = 0
df_ROI_FNA["Scan"] = 0
df_ROI_FNA_Sample = []
df_ROI_FNA_Row = []
df_ROI_FNA_Scan = []
for i in range(len(df_ROI_FNA["pixel_id"])):
    df_ROI_FNA_Sample.append("_".join(str(df_ROI_FNA["pixel_id"][i]).split("_")[:-2]))
    df_ROI_FNA_Row.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-1]))
    df_ROI_FNA_Scan.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-2]))
df_ROI_FNA["Sample"] = df_ROI_FNA_Sample
df_ROI_FNA["Row"] = df_ROI_FNA_Row
df_ROI_FNA["Scan"] = df_ROI_FNA_Scan
pop_column = df_ROI_FNA.pop('Scan')
df_ROI_FNA.insert(1, 'Scan', pop_column)
pop_column = df_ROI_FNA.pop('Row')
df_ROI_FNA.insert(1, 'Row', pop_column)
pop_column = df_ROI_FNA.pop('Sample')
df_ROI_FNA.insert(1, 'Sample', pop_column)

df_ROI_DirectIn["Sample"] = 0
df_ROI_DirectIn["Row"] = 0
df_ROI_DirectIn["Scan"] = 0
df_ROI_DirectIn_Sample = []
df_ROI_DirectIn_Row = []
df_ROI_DirectIn_Scan = []
for i in range(len(df_ROI_DirectIn["pixel_id"])):
    df_ROI_DirectIn_Sample.append("_".join(str(df_ROI_DirectIn["pixel_id"][i]).split("_")[:-2]))
    df_ROI_DirectIn_Row.append(int(str(df_ROI_DirectIn["pixel_id"][i]).split("_")[-1]))
    df_ROI_DirectIn_Scan.append(int(str(df_ROI_DirectIn["pixel_id"][i]).split("_")[-2]))
df_ROI_DirectIn["Sample"] = df_ROI_DirectIn_Sample
df_ROI_DirectIn["Row"] = df_ROI_DirectIn_Row
df_ROI_DirectIn["Scan"] = df_ROI_DirectIn_Scan
pop_column = df_ROI_DirectIn.pop('Scan')
df_ROI_DirectIn.insert(1, 'Scan', pop_column)
pop_column = df_ROI_DirectIn.pop('Row')
df_ROI_DirectIn.insert(1, 'Row', pop_column)
pop_column = df_ROI_DirectIn.pop('Sample')
df_ROI_DirectIn.insert(1, 'Sample', pop_column)

In [7]:
df_ROI = df_ROI.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_train = df_ROI_train.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_ext = df_ROI_ext.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_ingested = df_ROI_ingested.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_FNA = df_ROI_FNA.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_DirectIn = df_ROI_DirectIn.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [8]:
df_ROI_DirectIn

,pixel_id,Sample,Row,Scan,106.9792,108.0186,117.0458,124.0068,124.9906,125.0094,...,883.5338,885.5488,886.5521,887.5599,896.7424,919.7392,929.5816,1058.7522,1448.9673,type
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,2,31467.708984,0.000000,0.000000e+00,8226763.0,0.000000,68493.031250,...,235876.031250,1.331586e+06,637292.56250,216978.078125,0.00,0.0,0.000000,0.000000e+00,0.0,0
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,406,40128.769531,61065.281250,0.000000e+00,9441332.0,48337.761719,201059.593750,...,175070.093750,1.368040e+06,608271.62500,301507.562500,0.00,0.0,73137.265625,0.000000e+00,0.0,0
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,400,0.000000,47513.886719,9.613566e+05,8388430.0,0.000000,75890.343750,...,303005.050781,1.380952e+06,592850.56250,223408.312500,0.00,0.0,70446.976562,0.000000e+00,0.0,0
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,398,33994.718750,0.000000,0.000000e+00,7907643.5,42436.125000,165030.187500,...,228142.843750,1.385763e+06,796052.68750,286232.687500,0.00,0.0,0.000000,0.000000e+00,0.0,0
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,396,130806.023438,146788.015625,1.083952e+06,8832109.0,36234.144531,161709.421875,...,169939.171875,1.338164e+06,624783.25000,202964.859375,0.00,0.0,0.000000,4.198843e+07,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3023,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,245,196262.843750,62660.628906,0.000000e+00,15620852.0,238677.875000,249589.562500,...,261900.687500,1.095903e+06,646161.81250,262448.125000,0.00,0.0,0.000000,0.000000e+00,6885676.0,1
3024,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,244,197618.109375,86614.171875,0.000000e+00,19066292.0,204513.171875,254534.140625,...,66850.914062,9.897115e+05,589755.56250,254386.000000,2207757.25,0.0,0.000000,0.000000e+00,8082013.0,1
3025,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,242,217601.453125,314308.968750,0.000000e+00,19015252.0,68276.101562,315785.500000,...,196666.687500,1.093046e+06,470406.59375,249233.687500,2884882.75,0.0,64286.335938,0.000000e+00,8049726.0,1
3026,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,250,140621.875000,55656.457031,0.000000e+00,14194971.0,85350.992188,311581.562500,...,158502.265625,9.706004e+05,570611.75000,245216.609375,0.00,0.0,976041.125000,0.000000e+00,5692894.0,1


In [9]:
remove_features = []
for col in df_ROI.columns[4:-5]:
    if (list(df_ROI_FNA[col].describe())[-1] < 7000):
        remove_features.append(col)
for col in df_ROI.columns[-5:-1]:
    remove_features.append(col)

In [10]:
remove_features

['106.9792',
 '108.0186',
 '117.0458',
 '124.9906',
 '125.0094',
 '126.0017',
 '126.9036',
 '135.0298',
 '151.0214',
 '151.9976',
 '152.0018',
 '152.0281',
 '157.1228',
 '158.081',
 '161.045',
 '165.0327',
 '166.0168',
 '167.0199',
 '168.8356',
 '170.8327',
 '172.8294',
 '172.9903',
 '173.8302',
 '174.0219',
 '181.0496',
 '182.0106',
 '183.011',
 '186.0452',
 '186.9136',
 '188.9105',
 '196.0278',
 '196.0535',
 '200.0538',
 '201.0387',
 '202.0349',
 '202.1078',
 '209.1544',
 '218.1028',
 '219.0312',
 '225.0585',
 '230.0668',
 '233.0465',
 '240.9694',
 '242.176',
 '243.0622',
 '243.1792',
 '244.0654',
 '245.0411',
 '245.048',
 '247.0632',
 '251.1483',
 '254.034',
 '254.2207',
 '257.2394',
 '267.0734',
 '267.144',
 '267.2328',
 '268.0767',
 '269.0527',
 '270.2155',
 '271.0042',
 '271.0486',
 '272.2312',
 '274.0741',
 '277.0736',
 '279.0389',
 '279.054',
 '279.1622',
 '280.0425',
 '281.0351',
 '283.0669',
 '283.1376',
 '283.1916',
 '284.0717',
 '285.2435',
 '285.2708',
 '286.0604',
 '286.9

In [11]:
df_ROI = df_ROI.drop(columns = remove_features)
df_ROI_train = df_ROI_train.drop(columns = remove_features)
df_ROI_ext = df_ROI_ext.drop(columns = remove_features)
df_ROI_ingested = df_ROI_ingested.drop(columns = remove_features)
df_ROI_FNA = df_ROI_FNA.drop(columns = remove_features)
df_ROI_DirectIn = df_ROI_DirectIn.drop(columns = remove_features)

In [12]:
df_ROI_train

,pixel_id,Sample,Row,Scan,124.0068,179.0557,195.0506,215.0326,227.2016,233.1546,...,794.5239,857.5177,863.0912,865.5023,867.5128,883.5338,885.5488,886.5521,887.5599,type
0,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2,6,99,2239.475098,5285.062012,1357.171875,1765.094360,1203.201172,3446.257080,...,0.000000,4257.063965,0.0,0.000000,0.0,2001.672241,31168.623047,15905.403320,9028.520508,0
1,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2,65,33,3524.593506,10439.729492,2422.381836,2648.144775,1918.206421,2258.440186,...,0.000000,5939.440430,0.0,720.572388,0.0,3095.637207,47861.051636,24858.464844,15025.241211,0
2,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2,65,32,3002.329346,9840.478516,1611.806885,2219.107178,1318.495239,2668.108643,...,559.411255,6036.183594,0.0,1333.212646,0.0,4334.495117,51022.324219,28314.386719,13455.000000,0
3,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2,65,31,3151.715820,10352.171875,1822.328003,2033.376465,1164.436768,2560.143066,...,0.000000,7873.383301,0.0,1393.671265,0.0,4666.652344,55411.539062,30245.667969,15749.697266,0
4,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2,65,30,2823.335205,9091.886719,1311.989014,2211.169678,1475.280273,3043.447266,...,0.000000,6885.431152,0.0,0.000000,0.0,4310.644043,50457.800781,26692.949219,14910.863281,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90955,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW,27,64,2627.657959,7337.008789,1067.565186,2703.399902,0.000000,0.000000,...,0.000000,3075.371582,0.0,0.000000,0.0,2014.154785,21569.531250,11240.609375,5688.204102,1
90956,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW,27,63,2739.631104,8168.624023,1468.761353,2651.325195,0.000000,1742.902466,...,0.000000,3451.916016,0.0,0.000000,0.0,2887.587402,24355.515625,12368.116211,7241.357422,1
90957,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW,27,62,4019.153564,6650.159180,0.000000,4812.207520,0.000000,1897.142578,...,0.000000,2265.875732,0.0,0.000000,0.0,0.000000,17327.812500,9655.767578,5455.035645,1
90958,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW,27,32,3170.010010,6306.125000,1377.150757,4316.224609,830.407227,2658.239990,...,0.000000,3003.921631,0.0,1392.182739,0.0,1890.196899,21500.857422,9323.784180,5401.045410,1


In [13]:
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI = df_ROI.loc[df_ROI["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])


In [14]:
sample_ROI_list = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI]

In [15]:
name_list = ["HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI"]

In [16]:
sample_ROI_list_train = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI]

sample_ROI_list_ext = [HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI]

sample_ROI_list_ingested = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI]

sample_ROI_list_FNA = [HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI]

sample_ROI_list_DirectIn = [HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI]

In [17]:
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI

,pixel_id,Sample,Row,Scan,124.0068,179.0557,195.0506,215.0326,227.2016,233.1546,...,794.5239,857.5177,863.0912,865.5023,867.5128,883.5338,885.5488,886.5521,887.5599,type
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,2,8226763.0,0.0000,8.734356e+05,597479.68750,6.796848e+05,8.966212e+05,...,470031.562500,352169.218750,5.126044e+06,6.655439e+05,962239.578125,235876.031250,1331585.625,6.372926e+05,216978.078125,0
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,4,10210607.0,0.0000,1.142831e+06,351061.84375,9.616746e+05,9.388458e+05,...,389429.312500,396866.031250,4.936565e+06,4.599889e+05,78579.179688,196964.875000,1451490.000,5.922894e+05,350863.875000,0
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,5,11721988.0,598518.6875,1.298470e+06,543811.68750,1.237240e+06,1.225918e+06,...,630830.546875,356264.906250,0.000000e+00,8.741948e+05,257116.734375,319688.312500,1853836.250,9.405958e+05,273320.562500,0
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,9,11873774.0,0.0000,1.397608e+06,423693.03125,1.173549e+06,8.920201e+05,...,458401.031250,306777.687500,5.681472e+06,1.040019e+06,180495.234375,345452.562500,1594346.000,7.149868e+05,310046.593750,0
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,11,10862974.0,0.0000,1.406976e+06,391590.71875,1.114541e+06,1.054546e+06,...,416463.625000,254745.515625,5.246431e+06,5.643072e+05,192787.125000,246124.218750,1256030.875,6.319585e+05,204829.156250,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,595,9092751.0,0.0000,1.107328e+06,704786.37500,1.590076e+06,8.988187e+05,...,475201.468750,348169.343750,0.000000e+00,1.377972e+06,344897.656250,225340.187500,1579836.875,1.097872e+06,339542.750000,0
297,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,596,10097616.0,0.0000,1.002128e+06,332230.18750,1.343452e+06,8.545741e+05,...,467592.343750,349874.250000,6.924906e+06,6.244413e+05,235178.156250,270488.125000,1550177.125,6.325014e+05,268800.093750,0
298,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,598,10767637.0,0.0000,1.173724e+06,500021.65625,1.246096e+06,9.850924e+05,...,351638.843750,254313.562500,0.000000e+00,8.760864e+05,270689.937500,271963.906250,1346329.875,7.646786e+05,237992.156250,0
299,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,600,10176433.0,0.0000,9.392391e+05,327043.75000,1.076830e+06,8.701148e+05,...,582495.000000,491605.656250,0.000000e+00,8.865479e+05,186632.437500,273641.500000,1440054.750,9.782394e+05,301364.718750,0


In [18]:
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI

,pixel_id,Sample,Row,Scan,124.0068,179.0557,195.0506,215.0326,227.2016,233.1546,...,794.5239,857.5177,863.0912,865.5023,867.5128,883.5338,885.5488,886.5521,887.5599,type
0,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,1,12176738.0,0.000000,1728867.000,258694.984375,441864.50000,189055.109375,...,520098.109375,384737.875000,0.0,2.564622e+06,735014.500000,188071.890625,1.392550e+06,753209.37500,322619.656250,1
1,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,2,14135779.0,0.000000,2509547.750,251050.281250,524995.75000,195003.906250,...,718009.437500,205010.515625,0.0,1.715896e+06,592634.500000,197198.890625,1.173196e+06,534890.75000,58295.613281,1
2,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,11,11700827.0,190732.671875,2045673.625,258700.062500,361530.71875,161930.187500,...,490062.421875,203869.421875,0.0,1.845380e+06,511332.250000,92970.921875,8.755424e+05,527000.50000,136271.578125,1
3,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,14,11333077.0,0.000000,1513526.375,184270.093750,314123.25000,52518.691406,...,371046.339844,157180.796875,0.0,1.265864e+06,351378.718750,53579.789062,7.414910e+05,286182.34375,48463.089844,1
4,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,15,13524384.0,269026.531250,1983786.875,183153.671875,438186.06250,165417.375000,...,349871.164062,53684.167969,0.0,7.157385e+05,196756.578125,63260.312500,7.550344e+05,405735.18750,118373.390625,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,583,11499414.0,176470.250000,1430460.250,46794.984375,324425.06250,152517.484375,...,357059.265625,75734.007812,0.0,8.987371e+05,190348.093750,0.000000,7.573950e+05,162382.78125,0.000000,1
299,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,589,8563978.0,182286.593750,1462631.000,49274.761719,258134.03125,175414.015625,...,528942.468750,0.000000,0.0,1.339231e+06,560152.625000,131856.203125,8.186472e+05,383421.53125,230939.515625,1
300,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,602,10670922.0,0.000000,1559295.375,62621.281250,341265.59375,185168.984375,...,579655.984375,134935.500000,0.0,1.808776e+06,536243.500000,151135.359375,9.289883e+05,502995.84375,181200.140625,1
301,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S0...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01,1,604,12146854.0,149882.171875,1951225.250,132508.609375,335704.65625,159219.562500,...,577979.421875,302721.812500,0.0,2.496261e+06,736308.375000,184380.671875,1.370530e+06,650568.56250,293856.187500,1


# 2. Unsupervised Learning

## 2.1. Dataset Preparation

In [19]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [20]:
df_ROI_for_ML_Opti

,106.9792,108.0186,117.0458,124.0068,124.9906,125.0094,126.0017,126.9036,135.0298,151.0214,...,883.5338,885.5488,886.5521,887.5599,896.7424,919.7392,929.5816,1058.7522,1448.9673,type
pixel_id,,,,,,,,,,,,,,,,,,,,,
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_99_6,0.000000,0.000000,0.0,2.239475e+03,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,2001.672241,3.116862e+04,1.590540e+04,9028.520508,8.354571e+02,1208.605713,0.000000e+00,0.0,8.403403e+02,0
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_103_6,0.000000,0.000000,0.0,4.155391e+03,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,2424.003906,3.054501e+04,1.491937e+04,9713.387695,0.000000e+00,0.000000,0.000000e+00,0.0,0.000000e+00,0
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_41_7,0.000000,0.000000,0.0,1.292694e+03,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,2032.922119,3.321796e+04,1.514365e+04,9782.683594,9.260005e+02,1105.964844,0.000000e+00,0.0,0.000000e+00,0
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_42_7,0.000000,0.000000,0.0,1.242931e+03,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,2229.916992,3.551262e+04,1.685920e+04,9062.646484,6.865181e+02,1066.867798,0.000000e+00,0.0,6.715743e+02,0
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_98_7,0.000000,0.000000,0.0,2.048169e+03,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,2252.401855,3.354660e+04,1.740748e+04,8189.253418,1.019697e+03,849.780151,0.000000e+00,0.0,7.964874e+02,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_594_1,208032.921875,93289.890625,0.0,1.424341e+07,191679.906250,260365.359375,417522.50000,183456.328125,342139.187500,0.000000,...,340221.156250,1.470005e+06,7.135132e+05,518926.687500,4.410164e+06,0.000000,1.283354e+06,0.0,0.000000e+00,1
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_595_1,43703.371094,0.000000,0.0,7.453880e+06,71115.992188,135152.984375,280401.15625,78532.375000,142307.203125,111165.835938,...,294071.910156,2.130824e+06,1.099195e+06,437886.093750,0.000000e+00,0.000000,6.556245e+04,0.0,0.000000e+00,1
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_596_1,334999.531250,226093.421875,0.0,1.937670e+07,179186.796875,309036.312500,637430.31250,77071.039062,442702.937500,277333.437500,...,176609.109375,1.115554e+06,5.783931e+05,258386.812500,0.000000e+00,0.000000,0.000000e+00,0.0,7.080241e+06,1


## 2.2. PCA Analysis

In [21]:
shortlist = []
for col in list(df_ROI.columns):
    if col not in ['pixel_id', 'Sample', 'Row', 'Scan']:
        shortlist.append(col)

In [25]:
pd.DataFrame(shortlist).to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/shortlist_for_ML_Opti.csv")

In [22]:
df_ROI_for_ML_Opti = df_ROI_for_ML_Opti[shortlist]
df_ROI_for_ML_Opti_train = df_ROI_for_ML_Opti_train[shortlist]
df_ROI_for_ML_Opti_ext = df_ROI_for_ML_Opti_ext[shortlist]
df_ROI_for_ML_Opti_ingested = df_ROI_for_ML_Opti_ingested[shortlist]
df_ROI_for_ML_Opti_FNA = df_ROI_for_ML_Opti_FNA[shortlist]
df_ROI_for_ML_Opti_DirectIn = df_ROI_for_ML_Opti_DirectIn[shortlist]

In [23]:
df_ROI_for_ML_Opti

,124.0068,179.0557,195.0506,215.0326,227.2016,233.1546,241.2173,250.1449,253.2171,255.2328,...,794.5239,857.5177,863.0912,865.5023,867.5128,883.5338,885.5488,886.5521,887.5599,type
pixel_id,,,,,,,,,,,,,,,,,,,,,
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_99_6,2.239475e+03,5285.062012,1.357172e+03,1765.094360,1203.201172,3446.257080,1359.884644,10417.782227,9.099805e+03,2.593086e+04,...,0.000000,4257.063965,0.0,0.000000e+00,0.000000,2001.672241,3.116862e+04,1.590540e+04,9028.520508,0
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_103_6,4.155391e+03,6133.164062,1.921501e+03,2892.591553,1378.336182,5497.271484,1761.496948,19751.246094,1.075205e+04,2.775533e+04,...,0.000000,4487.459473,0.0,0.000000e+00,0.000000,2424.003906,3.054501e+04,1.491937e+04,9713.387695,0
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_41_7,1.292694e+03,3695.261230,0.000000e+00,2225.103027,1561.307861,6164.573242,1269.849487,14473.411133,1.494812e+04,3.460462e+04,...,0.000000,3653.516846,0.0,0.000000e+00,0.000000,2032.922119,3.321796e+04,1.514365e+04,9782.683594,0
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_42_7,1.242931e+03,3635.843750,4.887417e+02,2827.721436,1062.043701,5087.580078,1637.160278,13347.621094,1.188557e+04,3.105169e+04,...,584.368896,2427.404541,0.0,0.000000e+00,0.000000,2229.916992,3.551262e+04,1.685920e+04,9062.646484,0
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_98_7,2.048169e+03,5988.678223,8.096351e+02,1611.982788,1233.044678,2927.715088,1039.213745,9052.196289,9.366004e+03,2.587108e+04,...,0.000000,3240.175293,0.0,6.084071e+02,0.000000,2252.401855,3.354660e+04,1.740748e+04,8189.253418,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_594_1,1.424341e+07,557954.500000,3.205326e+06,796842.000000,495921.750000,219235.593750,338492.156250,0.000000,2.609134e+06,3.436813e+07,...,652214.281250,386555.968750,0.0,1.163364e+06,284302.468750,340221.156250,1.470005e+06,7.135132e+05,518926.687500,1
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_595_1,7.453880e+06,371035.187500,2.420200e+06,131669.671875,185862.343750,162270.281250,55366.816406,41896.089844,7.838960e+05,9.848581e+06,...,600926.062500,335784.593750,0.0,1.622990e+06,407330.113281,294071.910156,2.130824e+06,1.099195e+06,437886.093750,1
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_596_1,1.937670e+07,658093.375000,3.174590e+06,757211.875000,353896.843750,170094.687500,247064.609375,0.000000,1.619394e+06,2.317884e+07,...,358901.562500,541987.187500,0.0,5.276537e+05,193192.625000,176609.109375,1.115554e+06,5.783931e+05,258386.812500,1


In [26]:
df_ROI_for_ML_Opti.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/df_ROI_for_ML_Opti_raMSIn.csv")
df_ROI_for_ML_Opti_train.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/df_ROI_for_ML_Opti_train_raMSIn.csv")
df_ROI_for_ML_Opti_ext.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/df_ROI_for_ML_Opti_ext_raMSIn.csv")
df_ROI_for_ML_Opti_ingested.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/df_ROI_for_ML_Opti_ingested_raMSIn.csv")
df_ROI_for_ML_Opti_FNA.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/df_ROI_for_ML_Opti_FNA_raMSIn.csv")
df_ROI_for_ML_Opti_DirectIn.to_csv(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/df_ROI_for_ML_Opti_DirectIn_raMSIn.csv")

In [38]:
## PCA Analysis

def pca_visual(df=df_ROI_for_ML_Opti_DirectIn):

    pca = PCA(n_components=3)
    raMSI_ML_mz_df = df.drop("type", axis = 1)
    pca.fit(raMSI_ML_mz_df)
    result = pd.DataFrame(pca.transform(raMSI_ML_mz_df), columns=['PCA%i' % i for i in range(3)], index=df_ROI_for_ML_Opti_DirectIn.index)
    
    plt.subplots(dpi = 300)
    plt.scatter(result['PCA1'], result['PCA2'], c = df['type'], s=1)
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"I:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/raMSIn/PCA_DirectIn.tif", bbox_inches = 'tight')

    components = pca.fit_transform(raMSI_ML_mz_df)
    total_var = pca.explained_variance_ratio_.sum() * 100
    fig = px.scatter_3d(
        components, x=0, y=1, z=2, color=df['type'],
        title = f'Total Explained Variance: {total_var:.2f}%',
        labels = {'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
    )
    fig.show()

In [39]:
pca_visual()

# 3. Supervised Learning

## 3.1. Preparation of training and test dataset

In [23]:
def split_dataset(df=df_ROI_for_ML_Opti_ingested):
    raMSI_ML_NoType = df.drop("type", axis = 1)
    raMSI_ML_Type=df[['type']]
    X_train,X_test,y_train,y_test=train_test_split(raMSI_ML_NoType,raMSI_ML_Type,test_size=0.2,random_state=42)
    return X_train,X_test,y_train,y_test

In [24]:
X_train,X_test,y_train,y_test=split_dataset(df=df_ROI_for_ML_Opti_ingested)

In [25]:
X_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]]

In [26]:
y_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[-1]]

In [27]:
X_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[:-1]]

In [28]:
y_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[-1]]

In [29]:
X_DirectIn = df_ROI_for_ML_Opti_DirectIn[df_ROI_for_ML_Opti_DirectIn.columns[:-1]]

In [30]:
y_DirectIn = df_ROI_for_ML_Opti_DirectIn[df_ROI_for_ML_Opti_DirectIn.columns[-1]]

## 3.2 Machine Learning Modeling

In [31]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

### 3.2.2. XBG

In [32]:
## Build XGBoost Model
from copy import deepcopy
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix,roc_curve
import xgboost as xgb
import time
import warnings
mpl.rcParams['figure.dpi'] = 300

"""def XGBoost_Opt_func(XTrain=X_ingested, XTest=X_FNA, yTrain=y_ingested, yTest=y_FNA):
    import warnings
    warnings.filterwarnings('ignore')
    
    results_dict = {}
    
    model_xgb = xgb.XGBClassifier(
        objective='binary:hinge', 
        booster='gbtree', 
        eval_metric='auc', 
        tree_method='hist', 
        grow_policy='lossguide', 
        use_label_encoder=False)
    #n_estimators=100,objective= 'binary:hinge',use_label_encoder = False)
    model_xgb.fit(XTrain, yTrain)

    default_params = {}
    gparams = model_xgb.get_params()
    for key in gparams.keys():
        gp = gparams[key]
        default_params[key] = [gp]
    clf0 = GridSearchCV(
        estimator=model_xgb, 
        scoring='accuracy', 
        param_grid=default_params, 
        return_train_score=True, 
        verbose=1, cv=10)
    clf0.fit(XTrain, yTrain.values.ravel())
    
    df = pd.DataFrame(clf0.cv_results_)
    train_predictions = clf0.predict(XTrain)
    test_predictions = clf0.predict(XTest)
    cfm_train = confusion_matrix(yTrain, train_predictions)
    cfm_test = confusion_matrix(yTest, test_predictions)
    accs_train = accuracy_score(yTrain, train_predictions)
    accs_test = accuracy_score(yTest, test_predictions)
    f1s_train_p1 = f1_score(yTrain, train_predictions, pos_label=1)
    f1s_train_p0 = f1_score(yTrain, train_predictions, pos_label=0)
    f1s_test_p1 = f1_score(yTest, test_predictions, pos_label=1)
    f1s_test_p0 = f1_score(yTest, test_predictions, pos_label=0)
    test_ras = roc_auc_score(yTest, clf0.predict_proba(XTest)[:,1])
    bp = clf0.best_params_
    results_dict['model_xgb'] = {'iterable_parameter': np.nan, 
                             'classifier': deepcopy(clf0), 
                             'cv_results': df.copy(), 
                             'cfm_train': cfm_train, 
                             'cfm_test': cfm_test, 
                             'train_accuracy': accs_train, 
                             'test_accuracy': accs_test, 
                             'train F1-score label 1': f1s_train_p1, 
                             'train F1-score label 0': f1s_train_p0, 
                             'test F1-score label 1': f1s_test_p1, 
                             'test F1-score label 0': f1s_test_p0, 
                             'test roc auc score': test_ras, 
                             'best_params': bp}
    
    return results_dict['model_xgb']
    
results_dict = XGBoost_Opt_func(XTrain=X_ingested, XTest=X_FNA, yTrain=y_ingested, yTest=y_FNA)"""

"def XGBoost_Opt_func(XTrain=X_ingested, XTest=X_FNA, yTrain=y_ingested, yTest=y_FNA):\n    import warnings\n    warnings.filterwarnings('ignore')\n    \n    results_dict = {}\n    \n    model_xgb = xgb.XGBClassifier(\n        objective='binary:hinge', \n        booster='gbtree', \n        eval_metric='auc', \n        tree_method='hist', \n        grow_policy='lossguide', \n        use_label_encoder=False)\n    #n_estimators=100,objective= 'binary:hinge',use_label_encoder = False)\n    model_xgb.fit(XTrain, yTrain)\n\n    default_params = {}\n    gparams = model_xgb.get_params()\n    for key in gparams.keys():\n        gp = gparams[key]\n        default_params[key] = [gp]\n    clf0 = GridSearchCV(\n        estimator=model_xgb, \n        scoring='accuracy', \n        param_grid=default_params, \n        return_train_score=True, \n        verbose=1, cv=10)\n    clf0.fit(XTrain, yTrain.values.ravel())\n    \n    df = pd.DataFrame(clf0.cv_results_)\n    train_predictions = clf0.predict(XTr

results_dict

In [33]:
## Build XGBoost Model
import xgboost as xgb


def XGBoost_func(num, lr, est, colsample_bytree, gamma, max_depth, subsample, XTrain=X_ingested, XVal= X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal=y_FNA, yTest=y_DirectIn):
    
    model_xgb = xgb.XGBClassifier(
        base_score = 0.5, 
        booster = 'gbtree', 
        colsample_bylevel = 1, 
        colsample_bynode = 1, 
        colsample_bytree = colsample_bytree, 
        enable_categorical = False, 
        eval_metric = 'auc', 
        gamma = gamma, 
        gpu_id = -1, 
        grow_policy = 'lossguide', 
        importance_type = None, 
        learning_rate = lr, 
        max_bin = 256, 
        max_cat_to_onehot = 4, 
        max_delta_step = 0, 
        max_depth = max_depth, 
        max_leaves = 0, 
        min_child_weight = 1, 
        #missing = nan, 
        monotone_constraints = '()', 
        n_estimators = num, 
        n_jobs = 0, 
        num_parallel_tree = 1, 
        objective = 'binary:hinge', 
        predictor = 'auto', 
        random_state = 42, 
        reg_alpha = 0, 
        reg_lambda = 1, 
        sampling_method = 'uniform', 
        scale_pos_weight = None, 
        subsample = subsample, 
        tree_method = 'hist', 
        use_label_encoder = False, 
        validate_parameters = 1, 
        verbosity = None, 
        early_stopping_rounds = est)
    #n_estimators=100,objective= 'binary:hinge',use_label_encoder = False)
    model_xgb.fit(XTrain, yTrain, 
                  eval_set=[(XVal, yVal), (XTest, yTest)])

    X_train_predictions = model_xgb.predict(XTrain)
    predictions_val = model_xgb.predict(XVal)
    predictions_test = model_xgb.predict(XTest)

    a = metrics.roc_auc_score(yTrain, X_train_predictions)
    b = metrics.roc_auc_score(yVal, predictions_val)
    c = metrics.roc_auc_score(yTest, predictions_test)
    
    return a, b, c

In [44]:
#num_ = [25, 50, 75, 100] #4
#lr_ = [1, 2.5, 5, 7.5, 10]  #5
#est_ = [5, 10, 20]  #3
#col_bytree = [0.5, 0.75, 1]  # 3
#gamma_ = [0, 0.1, 1]  # 3
#max_depth_ = [3, 4, 5, 6]  # 4
#subsample_ = [0.5, 0.75, 1]  # 3
#num_ = [50]# 10, 15, 20, 25, 30, 35, 40, 45, 50, 75, 100]  #1
#lr_ = [1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8]  #14
#est_ = [5, 20]  #2
#col_bytree = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]  # 11
#gamma_ = [0]  # 1
#max_depth_ = [3, 4, 5]  # 3
#subsample_ = [0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]  # 13
#num_ = [75]#25, 35, 45, 55, 75, 100]  #6
#lr_ = [4, 4.5, 5, 5.5, 7.5, 10]  #6
#est_ = [10, 20]  #2
#col_bytree = [0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]  # 8
#gamma_ = [0, 0.2]  # 2
#max_depth_ = [3, 4, 5, 6]  # 4
#subsample_ = [0.4, 0.45, 0.5, 0.55, 0.6]  # 5

num_ = [25]#25, 35, 45, 55, 75, 100]  #6
lr_ = [1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 
       2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 
       3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, 
       4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5.0, 
       5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6.0, 
       6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9, 7.0, 
       7.1, 7.2, 7.3, 7.4, 7.5, 7.75, 8, 8.25, 8.5, 8.75, 9, 9.25, 9.5, 9.75, 10]  #80
est_ = [2, 5]  #2
col_bytree = [0.525, 0.55, 0.575]  # 3
gamma_ = [0, 0.2, 0.4]  # 3
max_depth_ = [4]  # 1
subsample_ = [0.775, 0.8, 0.825]  # 3

ntree = []
llrr = []
estest = []
col_ = []
g_ = []
max_d_ = []
ss_ = []
a_ = []
b_ = []
c_ = []
for num in num_:
    for lr in lr_:
        for est in est_:
            for colsample_bytree in col_bytree:
                for gamma in gamma_:
                    for max_depth in max_depth_:
                        for subsample in subsample_:
                            print(num, lr, est, colsample_bytree, gamma, max_depth, subsample)
                            a, b, c = XGBoost_func(num = num, lr=lr, est=est, colsample_bytree=colsample_bytree, gamma=gamma, max_depth=max_depth, subsample=subsample)
                            ntree.append(num)
                            llrr.append(lr)
                            estest.append(est)
                            col_.append(colsample_bytree)
                            g_.append(gamma)
                            max_d_.append(max_depth)
                            ss_.append(subsample)
                            a_.append(a)
                            b_.append(b)
                            c_.append(c)

25 1 2 0.525 0 4 0.775


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[0]	validation_0-auc:0.51614	validation_1-auc:0.50626
[1]	validation_0-auc:0.50229	validation_1-auc:0.54547
[2]	validation_0-auc:0.50052	validation_1-auc:0.50954
25 1 2 0.525 0 4 0.8


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[0]	validation_0-auc:0.51614	validation_1-auc:0.50626
[1]	validation_0-auc:0.50229	validation_1-auc:0.54547
[2]	validation_0-auc:0.50108	validation_1-auc:0.50954
25 1 2 0.525 0 4 0.825


...

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[0]	validation_0-auc:0.51614	validation_1-auc:0.50626
[1]	validation_0-auc:0.51049	validation_1-auc:0.48466
[2]	validation_0-auc:0.49616	validation_1-auc:0.50594
[3]	validation_0-auc:0.51669	validation_1-auc:0.47609
[4]	validation_0-auc:0.42233	validation_1-auc:0.54533
[5]	validation_0-auc:0.52506	validation_1-auc:0.47609
[6]	validation_0-auc:0.45219	validation_1-auc:0.47510
[7]	validation_0-auc:0.47846	validation_1-auc:0.49967
[8]	validation_0-auc:0.49108	validation_1-auc:0.46522
25 7.2 5 0.55 0.2 4 0.825


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[0]	validation_0-auc:0.51614	validation_1-auc:0.50626
[1]	validation_0-auc:0.50883	validation_1-auc:0.48466
[2]	validation_0-auc:0.69654	validation_1-auc:0.50561
[3]	validation_0-auc:0.53248	validation_1-auc:0.50956
[4]	validation_0-auc:0.52970	validation_1-auc:0.51022
[5]	validation_0-auc:0.51984	validation_1-auc:0.57817
[6]	validation_0-auc:0.50758	validation_1-auc:0.57751
[7]	validation_0-auc:0.50427	validation_1-auc:0.51286


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [45]:
df_lr_est = pd.DataFrame()
df_lr_est["ntree"] = ntree
df_lr_est["lr"] = llrr
df_lr_est["est"] = estest
df_lr_est["colsample_bytree"] = col_
df_lr_est["gamma"] = g_
df_lr_est["max_depth"] = max_d_
df_lr_est["subsample"] = ss_
df_lr_est["train"] = a_
df_lr_est["val"] = b_
df_lr_est["test"] = c_
df_lr_est.to_csv(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/XGB_Importance/new/df1_allpara6_25trees_shortlisted.csv")

In [33]:
X_FNA

,124.0068,179.0557,195.0506,215.0326,227.2016,233.1546,241.2173,250.1449,253.2171,255.2328,...,790.5404,794.5239,857.5177,863.0912,865.5023,867.5128,883.5338,885.5488,886.5521,887.5599
pixel_id,,,,,,,,,,,,,,,,,,,,,
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_4_1,4611.293457,3226.861328,3485.938232,3152.019287,0.000000,1176.927124,0.000000,5024.136719,1373.350342,6138.570312,...,0.000000,1056.883789,2552.779541,0.0,0.0,0.0,1796.736816,16540.734375,7923.897461,4551.694824
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_5_1,3961.981201,3259.049805,3002.470459,3041.586182,1012.966980,2934.085449,0.000000,7385.251953,2073.261719,10169.780273,...,0.000000,1557.148560,2602.783691,0.0,0.0,0.0,1665.447632,20512.451172,10123.107422,4599.267578
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_6_1,4367.012207,3726.879395,3826.441895,3117.169434,1271.352051,2626.856201,1117.611572,8289.688477,2017.503296,8510.083984,...,594.280090,1174.772339,2369.206055,0.0,0.0,0.0,1464.021362,19782.027344,9907.168945,5438.246582
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_194_1,11969.335938,9017.208008,6158.123535,6283.457520,7647.118164,10473.511719,7787.978027,27578.626953,14612.412109,83971.859375,...,1097.482422,834.205872,2010.571289,0.0,0.0,0.0,1961.100708,17403.191406,8705.579102,4691.113281
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_195_1,12198.677734,9426.558594,6647.423828,7803.814941,9712.125977,9492.361328,7533.042480,29367.962891,15570.219727,89553.609375,...,1770.806274,0.000000,2164.303955,0.0,0.0,0.0,1127.184814,17491.759766,8319.411133,5889.952148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_66_124,2865.312622,0.000000,0.000000,0.000000,6191.540527,4530.698242,6065.934082,17985.902344,7654.991699,21052.193359,...,0.000000,845.259277,2223.746582,0.0,0.0,0.0,1287.660767,13064.731445,7066.825684,3949.777100
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_60_125,1361.154175,0.000000,0.000000,0.000000,7211.597168,6650.379395,6936.910156,23960.953125,9142.936523,22014.380859,...,0.000000,0.000000,1802.681030,0.0,0.0,0.0,743.491882,11239.168945,5646.083984,3617.960449
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_63_125,1579.098389,495.335846,0.000000,0.000000,7218.175293,5047.635254,5822.427734,24248.208984,8050.921875,24665.642578,...,0.000000,849.761780,1675.243774,0.0,0.0,0.0,1382.497559,11548.824219,5129.648926,3673.355957


In [36]:
## Build XGBoost Model
import xgboost as xgb


def XGBoost_func(XTrain=X_ingested, XVal= X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal=y_FNA, yTest=y_DirectIn):
    
    model_xgb = xgb.XGBClassifier(
        base_score = 0.5, 
        booster = 'gbtree', 
        colsample_bylevel = 1, 
        colsample_bynode = 1, 
        colsample_bytree = 0.55, 
        enable_categorical = False, 
        eval_metric = 'auc', 
        gamma = 0, 
        gpu_id = -1, 
        grow_policy = 'lossguide', 
        importance_type = None, 
        learning_rate = 5.25, 
        max_bin = 256, 
        max_cat_to_onehot = 4, 
        max_delta_step = 0, 
        max_depth = 4, 
        max_leaves = 0, 
        min_child_weight = 1, 
        #missing = nan, 
        monotone_constraints = '()', 
        n_estimators = 25, 
        n_jobs = 0, 
        num_parallel_tree = 1, 
        objective = 'binary:hinge', 
        predictor = 'auto', 
        random_state = 42, 
        reg_alpha = 0, 
        reg_lambda = 1, 
        sampling_method = 'uniform', 
        scale_pos_weight = None, 
        subsample = 0.8, 
        tree_method = 'hist', 
        use_label_encoder = False, 
        validate_parameters = 1, 
        verbosity = None, 
        early_stopping_rounds = 2)
    #n_estimators=100,objective= 'binary:hinge',use_label_encoder = False)
    model_xgb.fit(XTrain, yTrain, 
                  eval_set=[(XVal, yVal), (XTest, yTest)])

    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_xgb.predict(XTrain)
    print("Training Set AUROC:",metrics.roc_auc_score(yTrain, X_train_predictions))
    print(classification_report(yTrain,X_train_predictions))
    print(confusion_matrix(yTrain,X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yTrain, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/XGB_Importance/new/XGB_ROC_train_shisted.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val = model_xgb.predict(XVal)
    print("Validation Set AUROC:",metrics.roc_auc_score(yVal, predictions_val))
    print(classification_report(yVal,predictions_val))
    print(confusion_matrix(yVal,predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/XGB_Importance/new/XGB_ROC_val_shoisted.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test set
    predictions_test = model_xgb.predict(XTest)
    print("Test Set AUROC:",metrics.roc_auc_score(yTest, predictions_test))
    print(classification_report(yTest,predictions_test))
    print(confusion_matrix(yTest,predictions_test))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_test, tpr_test, _ = metrics.roc_curve(yTest, predictions_test)
    plt.plot(fpr_test,tpr_test)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/XGB_Importance/new/XGB_ROC_test_shosted.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the important features (top 20 are shown)
    feature_imp = pd.Series(model_xgb.feature_importances_,index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp[:25], y=feature_imp[:25].index)
    plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/XGB_Importance/new/XGB_TopFeatures_stlisted.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the boxplot of the important features between the two groups
    #fig = px.box(data_frame = df_ROI_for_ML_Opti,x = feature_imp[:20].index,color= 'type')
    #fig.show()
    
    return x, y

In [37]:
coeff_XGB, TOPfeatures_XGB = XGBoost_func()

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[0]	validation_0-auc:0.51614	validation_1-auc:0.50626
[1]	validation_0-auc:0.50982	validation_1-auc:0.54928
[2]	validation_0-auc:0.68803	validation_1-auc:0.66834
[3]	validation_0-auc:0.51641	validation_1-auc:0.51061
Training Set AUROC: 0.9068540953356574
              precision    recall  f1-score   support

           0       0.97      0.84      0.90     50392
           1       0.85      0.97      0.91     46643

    accuracy                           0.90     97035
   macro avg       0.91      0.91      0.90     97035
weighted avg       0.91      0.90      0.90     97035

[[42545  7847]
 [ 1426 45217]]
Validation Set AUROC: 0.6880301387195628
              precision    recall  f1-score   support

           0       0.66      0.77      0.71     44540
           1       0.72      0.61      0.66     44161

    accuracy                           0.69     88701
   macro avg       0.69      0.69      0.69     88701
weighted avg       0.69      0.69      0.69     88701

[[34175 10365]
 [17

In [38]:
coeff_XGB  # coeff

311.1684    0.472777
269.2486    0.180840
215.0326    0.069475
295.2278    0.035886
883.5338    0.031075
              ...   
328.2365    0.000000
329.2492    0.000000
351.2182    0.000000
436.1052    0.000000
887.5599    0.000000
Length: 84, dtype: float32

In [39]:
TOPfeatures_XGB  # features

Index(['311.1684', '269.2486', '215.0326', '295.2278', '883.5338', '309.1704',
       '738.5059', '435.2965', '280.2361', '241.2173', '311.2228', '339.1996',
       '353.1999', '325.1842', '250.1449', '514.2846', '265.1478', '233.1546',
       '867.5128', '227.2016', '857.5177', '337.2045', '794.5239', '303.2328',
       '179.0557', '790.5404', '253.2171', '666.5112', '602.4456', '281.2484',
       '438.9716', '279.2328', '592.994', '664.9179', '563.9381', '515.2877',
       '124.0068', '665.7769', '717.4791', '747.5', '748.5054', '760.5032',
       '762.5074', '509.2891', '766.539', '767.5368', '770.5046', '773.5333',
       '863.0912', '865.5023', '885.5488', '886.5521', '763.511', '327.2328',
       '485.2796', '480.3096', '195.0506', '255.2328', '256.2363', '266.1513',
       '267.1965', '269.2122', '271.2278', '277.2173', '282.2519', '283.2279',
       '283.2642', '284.2676', '285.2071', '293.1774', '297.1155', '297.153',
       '297.2435', '298.2469', '299.2592', '301.2174', '304

In [40]:
len(TOPfeatures_XGB)

84

In [38]:
TOP_features_XGB  # old features (dont run)

NameError: name 'TOP_features_XGB' is not defined

## 3.3. Sensitivity

In [41]:
def df_maker(features_list, df= df_ROI_for_ML_Opti_ingested):
    df_sensitivity = df.T.T.drop(columns=list(df.columns))
    for fea in features_list:
        df_sensitivity[fea] = df[fea]
    df_sensitivity["type"] = df["type"]
    return df_sensitivity

In [42]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 3000x3000 with 0 Axes>

### 3.3.2. XGB

In [43]:
def XGBoost_sensitivity(XTrain=X_ingested, XVal=X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal=y_FNA, yTest=y_DirectIn):
    
    model_xgb = xgb.XGBClassifier(
        base_score = 0.5, 
        booster = 'gbtree', 
        colsample_bylevel = 1, 
        colsample_bynode = 1, 
        colsample_bytree = 0.55, 
        enable_categorical = False, 
        eval_metric = 'auc', 
        gamma = 0, 
        gpu_id = -1, 
        grow_policy = 'lossguide', 
        importance_type = None, 
        learning_rate = 5.25, 
        max_bin = 256, 
        max_cat_to_onehot = 4, 
        max_delta_step = 0, 
        max_depth = 4, 
        max_leaves = 0, 
        min_child_weight = 1, 
        #missing = nan, 
        monotone_constraints = '()', 
        n_estimators = 25, 
        n_jobs = 0, 
        num_parallel_tree = 1, 
        objective = 'binary:hinge', 
        predictor = 'auto', 
        random_state = 42, 
        reg_alpha = 0, 
        reg_lambda = 1, 
        sampling_method = 'uniform', 
        scale_pos_weight = None, 
        subsample = 0.8, 
        tree_method = 'hist', 
        use_label_encoder = False, 
        validate_parameters = 1, 
        verbosity = None, 
        early_stopping_rounds = 2
    )
    
    #model_xgb.fit(XTrain, yTrain)
    model_xgb.fit(XTrain, yTrain, 
                  eval_set=[(XVal, yVal), (XTest, yTest)])

    fit_params = {'eval_set': [[XVal, yVal], [XTest, yTest]]}

    scores_XGB = cross_val_score(model_xgb, XTrain, yTrain, cv=5, 
                                 scoring = 'roc_auc',
                                 fit_params = fit_params)
    
    print(scores_XGB.mean())
    print(scores_XGB.std())
    X_train_predictions = model_xgb.predict(XTrain)
    predictions_val = model_xgb.predict(XVal)
    predictions_test = model_xgb.predict(XTest)
    
    a_f1 = f1_score(yTrain, X_train_predictions, pos_label=1)
    b_f1 = f1_score(yVal, predictions_val, pos_label=1)
    c_f1 = f1_score(yTest, predictions_test, pos_label=1)
    a_auc = metrics.roc_auc_score(yTrain, X_train_predictions)
    b_auc = metrics.roc_auc_score(yVal, predictions_val)
    c_auc = metrics.roc_auc_score(yTest, predictions_test)

    cv_acr_XGB = scores_XGB.mean()
    cv_std_XGB = scores_XGB.std()

    return cv_acr_XGB, cv_std_XGB, a_f1, b_f1, c_f1, a_auc, b_auc, c_auc

In [44]:
X_ingested

,124.0068,179.0557,195.0506,215.0326,227.2016,233.1546,241.2173,250.1449,253.2171,255.2328,...,790.5404,794.5239,857.5177,863.0912,865.5023,867.5128,883.5338,885.5488,886.5521,887.5599
pixel_id,,,,,,,,,,,,,,,,,,,,,
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_99_6,2239.475098,5285.062012,1357.171875,1765.094360,1203.201172,3446.257080,1359.884644,10417.782227,9099.804688,25930.855469,...,1250.714233,0.000000,4257.063965,0.0,0.000000,0.0,2001.672241,31168.623047,15905.403320,9028.520508
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_103_6,4155.390625,6133.164062,1921.501221,2892.591553,1378.336182,5497.271484,1761.496948,19751.246094,10752.050781,27755.326172,...,1064.463623,0.000000,4487.459473,0.0,0.000000,0.0,2424.003906,30545.011719,14919.373047,9713.387695
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_41_7,1292.693970,3695.261230,0.000000,2225.103027,1561.307861,6164.573242,1269.849487,14473.411133,14948.118164,34604.625000,...,820.255554,0.000000,3653.516846,0.0,0.000000,0.0,2032.922119,33217.960938,15143.654297,9782.683594
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_42_7,1242.931152,3635.843750,488.741669,2827.721436,1062.043701,5087.580078,1637.160278,13347.621094,11885.568359,31051.693359,...,1733.640259,584.368896,2427.404541,0.0,0.000000,0.0,2229.916992,35512.621094,16859.195312,9062.646484
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_98_7,2048.168701,5988.678223,809.635071,1611.982788,1233.044678,2927.715088,1039.213745,9052.196289,9366.003906,25871.083984,...,1646.036011,0.000000,3240.175293,0.0,608.407104,0.0,2252.401855,33546.597656,17407.480469,8189.253418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_37_30,424.004242,580.648621,0.000000,0.000000,2522.145996,3230.580566,1575.619995,9511.707031,663.538635,17844.457031,...,0.000000,0.000000,311.449890,0.0,0.000000,0.0,0.000000,6123.190430,2633.123535,1756.053955
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_38_30,595.734192,326.932434,240.059250,0.000000,1978.850342,3854.620850,1554.890381,9744.109375,635.373474,20401.718750,...,430.207520,0.000000,1771.128540,0.0,529.190857,0.0,1543.361328,20750.964844,10420.350586,5912.117676
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_39_30,284.465607,540.562500,0.000000,0.000000,2322.719727,3062.539062,1245.831055,9213.754883,311.672241,17421.740234,...,739.540833,0.000000,713.486450,0.0,0.000000,0.0,502.586395,12533.286133,7673.676270,3264.151123


In [45]:
len(list(df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]].columns))

84

In [46]:
cv_auc_his_XGB = []
cv_change_XGB = [0]
cv_std_his_XGB = []
a_f1_ = []
b_f1_ = []
c_f1_ = []
a_auc_ = []
b_auc_ = []
c_auc_ = []
f1_change_XGB = [0]
auc_change_XGB = [0]
f_ = [""]

features_list = list(df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]].columns)
X_ingested = df_ROI_for_ML_Opti_ingested[features_list]
X_FNA = df_ROI_for_ML_Opti_FNA[features_list]
X_DirectIn = df_ROI_for_ML_Opti_DirectIn[features_list]
cv_auc_XGB, cv_std_XGB, a_f1, b_f1, c_f1, a_auc, b_auc, c_auc = XGBoost_sensitivity(XTrain=X_ingested, XVal=X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal = y_FNA, yTest=y_DirectIn)

cv_auc_his_XGB.append(cv_auc_XGB)
cv_std_his_XGB.append(cv_std_XGB)

a_f1_.append(a_f1)
b_f1_.append(b_f1)
c_f1_.append(c_f1)

a_auc_.append(a_auc)
b_auc_.append(b_auc)
c_auc_.append(c_auc)

#for i in range(39):
for i in range(len(TOPfeatures_XGB)):
    print(i)
    features_list = list(df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]].columns)
    features_list.remove(list(TOPfeatures_XGB)[i])
    f_.append(list(TOPfeatures_XGB)[i])
    X_ingested = df_ROI_for_ML_Opti_ingested[features_list]
    X_FNA = df_ROI_for_ML_Opti_FNA[features_list]
    X_DirectIn = df_ROI_for_ML_Opti_DirectIn[features_list]
    cv_auc_XGB, cv_std_XGB, a_f1, b_f1, c_f1, a_auc, b_auc, c_auc = XGBoost_sensitivity(XTrain=X_ingested, XVal=X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal = y_FNA, yTest=y_DirectIn)
    
    cv_auc_his_XGB.append(cv_auc_XGB)
    cv_std_his_XGB.append(cv_std_XGB)

    a_f1_.append(a_f1)
    b_f1_.append(b_f1)
    c_f1_.append(c_f1)
    
    a_auc_.append(a_auc)
    b_auc_.append(b_auc)
    c_auc_.append(c_auc)
    

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[0]	validation_0-auc:0.51614	validation_1-auc:0.50626
[1]	validation_0-auc:0.50982	validation_1-auc:0.54928
[2]	validation_0-auc:0.68803	validation_1-auc:0.66834
[3]	validation_0-auc:0.51641	validation_1-auc:0.51061


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[0]	validation_0-auc:0.50124	validation_1-auc:0.50626
[1]	validation_0-auc:0.49541	validation_1-auc:0.50560
[2]	validation_0-auc:0.63006	validation_1-auc:0.50297


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


...

[0]	validation_0-auc:0.47133	validation_1-auc:0.43802
[1]	validation_0-auc:0.42532	validation_1-auc:0.43835
[2]	validation_0-auc:0.46676	validation_1-auc:0.45150
[3]	validation_0-auc:0.41396	validation_1-auc:0.45150
[4]	validation_0-auc:0.49462	validation_1-auc:0.44067


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[0]	validation_0-auc:0.47132	validation_1-auc:0.50659
[1]	validation_0-auc:0.48491	validation_1-auc:0.50593
[2]	validation_0-auc:0.50087	validation_1-auc:0.50626


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[0]	validation_0-auc:0.48399	validation_1-auc:0.50626
[1]	validation_0-auc:0.48397	validation_1-auc:0.57151
[2]	validation_0-auc:0.43897	validation_1-auc:0.57387
[3]	validation_0-auc:0.49939	validation_1-auc:0.54850
[4]	validation_0-auc:0.49346	validation_1-auc:0.57420
[5]	validation_0-auc:0.49919	validation_1-auc:0.54850
0.8977113759277472
0.07379094650872821


In [47]:
df_xgb_TOPfeatures = pd.DataFrame()
df_xgb_TOPfeatures["cv_auc_his_XGB"] = cv_auc_his_XGB
df_xgb_TOPfeatures["cv_std_his_XGB"] = cv_std_his_XGB
df_xgb_TOPfeatures["train_auc"] = a_auc_
df_xgb_TOPfeatures["val_auc"] = b_auc_
df_xgb_TOPfeatures["test_auc"] = c_auc_
df_xgb_TOPfeatures["train_f1"] = a_f1_
df_xgb_TOPfeatures["val_f1"] = b_f1_
df_xgb_TOPfeatures["test_f1"] = c_f1_
df_xgb_TOPfeatures["features"] = f_
df_xgb_TOPfeatures.to_csv(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/XGB_Importance/new/df_xgb_TOPfeatures.csv")

In [54]:
len(TOP_features_XGB)

84

In [55]:
cv_auc_his_XGB = []
cv_change_XGB = [0]
cv_std_his_XGB = []
a_f1_ = []
b_f1_ = []
c_f1_ = []
a_auc_ = []
b_auc_ = []
c_auc_ = []
f1_change_XGB = [0]
auc_change_XGB = [0]

features_list = list(df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]].columns)
X_ingested = df_ROI_for_ML_Opti_ingested[features_list]
X_FNA = df_ROI_for_ML_Opti_FNA[features_list]
X_DirectIn = df_ROI_for_ML_Opti_DirectIn[features_list]
cv_auc_XGB, cv_std_XGB, a_f1, b_f1, c_f1, a_auc, b_auc, c_auc = XGBoost_sensitivity(XTrain=X_ingested, XVal=X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal = y_FNA, yTest=y_DirectIn)

cv_auc_his_XGB.append(cv_auc_XGB)
cv_std_his_XGB.append(cv_std_XGB)

a_f1_.append(a_f1)
b_f1_.append(b_f1)
c_f1_.append(c_f1)

a_auc_.append(a_auc)
b_auc_.append(b_auc)
c_auc_.append(c_auc)

for i in range(len(features_list)-1):
    print(i)
    #features_list = list(TOPfeatures_XGB)[0:i+1]
    #features_list = list(TOP_features_XGB)#[0:i+1]
    features_list.remove(list(TOPfeatures_XGB)[-(i+1)])
    
    #df_sensitivity = df_maker(features_list=features_list, df= df_ROI_for_ML_Opti_ingested)
    #X_train, X_test, y_train, y_test = split_dataset(df= df_sensitivity)
    X_ingested = df_ROI_for_ML_Opti_ingested[features_list]
    X_FNA = df_ROI_for_ML_Opti_FNA[features_list]
    X_DirectIn = df_ROI_for_ML_Opti_DirectIn[features_list]
    #train_acr, test_acr = XGBoost_sensitivity(XTrain=X_train,XTest=X_test,yTrain=y_train,yTest=y_test)
    cv_auc_XGB, cv_std_XGB, a_f1, b_f1, c_f1, a_auc, b_auc, c_auc = XGBoost_sensitivity(XTrain=X_ingested, XVal=X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal = y_FNA, yTest=y_DirectIn)
    #acr_his_XGB.append(train_acr)
    cv_auc_his_XGB.append(cv_auc_XGB)
    #acr_his_XGB.append(test_acr)
    cv_std_his_XGB.append(cv_std_XGB)
    #print(acr_his_XGB[-2:])
    a_f1_.append(a_f1)
    b_f1_.append(b_f1)
    c_f1_.append(c_f1)
    a_auc_.append(a_auc)
    b_auc_.append(b_auc)
    c_auc_.append(c_auc)
    try:
        #print(((acr_his_XGB[-2]-acr_his_XGB[-4])/acr_his_XGB[-4])*100)
        cv_change_XGB.append(((cv_auc_his_XGB[-1]-cv_auc_his_XGB[0])/cv_auc_his_XGB[0])*100)
        #print(((acr_his_XGB[-1]-acr_his_XGB[-3])/acr_his_XGB[-3])*100)
        auc_change_XGB.append(((c_auc_[-1]-c_auc_[0])/c_auc_[0])*100)
        f1_change_XGB.append(((c_f1_[-1]-c_f1_[0])/c_f1_[0])*100)
    except:
        pass

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[0]	validation_0-auc:0.51614	validation_1-auc:0.50626
[1]	validation_0-auc:0.50982	validation_1-auc:0.54928
[2]	validation_0-auc:0.68803	validation_1-auc:0.66834
[3]	validation_0-auc:0.51641	validation_1-auc:0.51061
[4]	validation_0-auc:0.50902	validation_1-auc:0.61806


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[0]	validation_0-auc:0.50124	validation_1-auc:0.50626
[1]	validation_0-auc:0.49541	validation_1-auc:0.50560
[2]	validation_0-auc:0.63006	validation_1-auc:0.50297


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


...

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[0]	validation_0-auc:0.50062	validation_1-auc:0.50000
[1]	validation_0-auc:0.49782	validation_1-auc:0.50000
[2]	validation_0-auc:0.50062	validation_1-auc:0.50000
[0]	validation_0-auc:0.50062	validation_1-auc:0.50000
[1]	validation_0-auc:0.49777	validation_1-auc:0.50000
[2]	validation_0-auc:0.50392	validation_1-auc:0.50000
[0]	validation_0-auc:0.50227	validation_1-auc:0.50000
[1]	validation_0-auc:0.49749	validation_1-auc:0.50000


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[0]	validation_0-auc:0.50062	validation_1-auc:0.50000
[1]	validation_0-auc:0.49749	validation_1-auc:0.50000
[2]	validation_0-auc:0.50189	validation_1-auc:0.50000
[0]	validation_0-auc:0.50283	validation_1-auc:0.50000
[1]	validation_0-auc:0.49795	validation_1-auc:0.50000
[2]	validation_0-auc:0.50177	validation_1-auc:0.50000
[0]	validation_0-auc:0.50085	validation_1-auc:0.50000
[1]	validation_0-auc:0.49907	validation_1-auc:0.50000
[2]	validation_0-auc:0.50093	validation_1-auc:0.50000
0.8075271395258785
0.1479920304702643


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [56]:
df_xgb_TOPfeatures = pd.DataFrame()
df_xgb_TOPfeatures["cv_auc_his_XGB"] = cv_auc_his_XGB
df_xgb_TOPfeatures["cv_std_his_XGB"] = cv_std_his_XGB
df_xgb_TOPfeatures["cv_change_XGB"] = cv_change_XGB
df_xgb_TOPfeatures["train_auc"] = a_auc_
df_xgb_TOPfeatures["val_auc"] = b_auc_
df_xgb_TOPfeatures["test_auc"] = c_auc_
df_xgb_TOPfeatures["train_f1"] = a_f1_
df_xgb_TOPfeatures["val_f1"] = b_f1_
df_xgb_TOPfeatures["test_f1"] = c_f1_
df_xgb_TOPfeatures["c_change_XGB_auc"] = auc_change_XGB
df_xgb_TOPfeatures["c_change_XGB_f1"] = f1_change_XGB
df_xgb_TOPfeatures.to_csv(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/XGB_Importance/new/df_xgb_TOPfeatures_shortlisted.csv")

In [57]:
print(len(cv_auc_his_XGB), len(cv_std_his_XGB), len(cv_change_XGB), len(auc_change_XGB))

84 84 84 84


In [72]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 3000x3000 with 0 Axes>

In [76]:
M_new_vec2 = np.array(cv_auc_his_XGB[::-1])
Sigma_new_vec2 = np.array(cv_std_his_XGB[::-1])

lower_bound2 = M_new_vec2 - Sigma_new_vec2
upper_bound2 = M_new_vec2 + Sigma_new_vec2
x_axis2 = np.arange(84)

plt.plot(x_axis2, M_new_vec2, 'go--', linewidth=0.5, markersize=0.1)
plt.fill_between(x_axis2, lower_bound2, upper_bound2, alpha=.3)
plt.plot(16, cv_auc_his_XGB[::-1][16], marker="o", markersize=5, markeredgecolor="blue", markerfacecolor="red")
plt.text(16, cv_auc_his_XGB[::-1][16], "the 17th feature", fontdict=dict(color="black", alpha=0.8, size=18))
plt.xlabel("Number of Features", size=16)
plt.ylabel("Average AUROC Score", size=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylim((0,1))
plt.xlim((1,84))

plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/XGB_Importance/new/XGB_CV_Sensitivity.tif", bbox_inches = 'tight', dpi = 300)
plt.close()

In [82]:
#beCareful
TOP_features_XGB = list(TOPfeatures_XGB)[:17]

In [83]:
len(TOP_features_XGB)

17

In [84]:
TOP_features_XGB

['311.1684',
 '269.2486',
 '215.0326',
 '295.2278',
 '883.5338',
 '309.1704',
 '738.5059',
 '435.2965',
 '280.2361',
 '241.2173',
 '311.2228',
 '339.1996',
 '353.1999',
 '325.1842',
 '250.1449',
 '514.2846',
 '265.1478']

In [85]:
features_list_XGB = list(TOP_features_XGB)

In [90]:
df_sensitivity_XGB = df_maker(features_list=features_list_XGB, df= df_ROI_for_ML_Opti_ingested)
X_train, X_test, y_train, y_test = split_dataset(df= df_sensitivity_XGB)

In [91]:
X_ingested = df_ROI_for_ML_Opti_ingested[features_list_XGB]

In [92]:
X_FNA = df_ROI_for_ML_Opti_FNA[features_list_XGB]

In [93]:
X_DirectIn = df_ROI_for_ML_Opti_DirectIn[features_list_XGB]

In [94]:
## Build XGBoost Model
import xgboost as xgb


def XGBoost_func(XTrain=X_ingested, XVal=X_FNA, XTest=X_DirectIn, yTrain=y_ingested, yVal=y_FNA, yTest=y_DirectIn):
    
    model_xgb = xgb.XGBClassifier(
        base_score = 0.5, 
        booster = 'gbtree', 
        colsample_bylevel = 1, 
        colsample_bynode = 1, 
        colsample_bytree = 0.55, 
        enable_categorical = False, 
        eval_metric = 'auc', 
        gamma = 0, 
        gpu_id = -1, 
        grow_policy = 'lossguide', 
        importance_type = None, 
        learning_rate = 5.25, 
        max_bin = 256, 
        max_cat_to_onehot = 4, 
        max_delta_step = 0, 
        max_depth = 4, 
        max_leaves = 0, 
        min_child_weight = 1, 
        #missing = nan, 
        monotone_constraints = '()', 
        n_estimators = 25, 
        n_jobs = 0, 
        num_parallel_tree = 1, 
        objective = 'binary:hinge', 
        predictor = 'auto', 
        random_state = 42, 
        reg_alpha = 0, 
        reg_lambda = 1, 
        sampling_method = 'uniform', 
        scale_pos_weight = None, 
        subsample = 0.8, 
        tree_method = 'hist', 
        use_label_encoder = False, 
        validate_parameters = 1, 
        verbosity = None, 
        early_stopping_rounds = 2
    )
    

    #n_estimators=100,objective= 'binary:hinge',use_label_encoder = False)
    model_xgb.fit(XTrain, yTrain, 
                  eval_set=[(XVal, yVal), (XTest, yTest)])

    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_xgb.predict(XTrain)
    print("Training Set Accuracy:",metrics.roc_auc_score(yTrain, X_train_predictions))
    print(classification_report(yTrain,X_train_predictions))
    print(confusion_matrix(yTrain,X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yTrain, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/XGB_Importance/new/XGB_ROC_train2.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val = model_xgb.predict(XVal)
    print("Validation Set Accuracy:",metrics.roc_auc_score(yVal, predictions_val))
    print(classification_report(yVal,predictions_val))
    print(confusion_matrix(yVal,predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/XGB_Importance/new/XGB_ROC_val2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test set
    predictions_FNA = model_xgb.predict(XTest)
    print("FNA Test Set Accuracy:",metrics.roc_auc_score(yTest, predictions_FNA))
    print(classification_report(yTest,predictions_FNA))
    print(confusion_matrix(yTest,predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/XGB_Importance/new/XGB_ROC_test2.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the important features (top 20 are shown)
    feature_imp = pd.Series(model_xgb.feature_importances_,index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/XGB_Importance/new/XGB_TopFeatures2.tif", bbox_inches = 'tight')
    #plt.show()


In [95]:
XGBoost_func()

[0]	validation_0-auc:0.50144	validation_1-auc:0.50626


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[1]	validation_0-auc:0.52029	validation_1-auc:0.49835
[2]	validation_0-auc:0.52879	validation_1-auc:0.50626
Training Set Accuracy: 0.9492446932533724
              precision    recall  f1-score   support

           0       0.96      0.94      0.95     50392
           1       0.94      0.96      0.95     46643

    accuracy                           0.95     97035
   macro avg       0.95      0.95      0.95     97035
weighted avg       0.95      0.95      0.95     97035

[[47351  3041]
 [ 1920 44723]]
Validation Set Accuracy: 0.5014368104860866
              precision    recall  f1-score   support

           0       0.50      1.00      0.67     44540
           1       0.69      0.01      0.01     44161

    accuracy                           0.50     88701
   macro avg       0.59      0.50      0.34     88701
weighted avg       0.59      0.50      0.34     88701

[[44434   106]
 [43929   232]]
FNA Test Set Accuracy: 0.5062636468537799
              precision    recall  f1-score   su

In [96]:
noSTDtop16 = pd.DataFrame()
noSTDtop16["top"] = list(features_list_XGB)
noSTDtop16.to_csv(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/noSTDtop17.csv")

In [21]:
TOPfeatures_XGB_ = list(pd.read_csv(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/noSTDtop17.csv")["top"])
TOPfeatures_XGB = []
for ion in TOPfeatures_XGB_:
    TOPfeatures_XGB.append(str(ion))

In [98]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 3000x3000 with 0 Axes>

In [99]:
def genAROItif(sample_ROI, sample_, name, mz, vmin_, vmax_):

    df_heatmap = sample_ROI[["Row", "Scan", mz]]
    df_heatmap2 = pd.DataFrame(np.array([[0, 0, 0]]), columns=['Row', 'Scan', mz]).reset_index().drop(columns = ["index"])
    df_HM = df_heatmap2.loc[df_heatmap["Row"] == 0].loc[df_heatmap2["Scan"] == 0]
    Heat_Row = sample_["Row"]
    Heat_Scan = sample_["Scan"]


    #hi = 0
    for y in range(len(set(Heat_Row))):
        for x in range(1, len(set(Heat_Scan))+1):

            df_in = df_heatmap.loc[df_heatmap["Row"] == (y+1)].loc[df_heatmap["Scan"] == x]
            if len(df_in) == 1:
                df_HM = pd.concat([df_HM, df_in], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


            elif len(df_in) == 0:
                df_heatmap2["Row"] = y+1
                df_heatmap2["Scan"] = x
                df_HM = pd.concat([df_HM, df_heatmap2], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


    #print(name, mz, hi, len(df_HM))
    if "index" in df_HM.columns:
        df_HM = df_HM.drop(columns = ["index", "Row", "Scan"])
    else:
        df_HM = df_HM.drop(columns = ["Row", "Scan"])

    df_HM = df_HM.T
    New_y = []
    for y in range(1, len(set(Heat_Row))+1):
        New_y.append(str(y))
    New_x = []
    for x in range(1, len(set(Heat_Scan))+1):
        New_x.append(str(x))

    arr = df_HM.values.copy()
    arr.resize(len(set(Heat_Row)), len(set(Heat_Scan)))

    df_HM = pd.DataFrame(arr)
    df_HM.index = New_y
    df_HM.columns = New_x

    for i in list(df_HM.columns):
        df_HM[i] = df_HM[i].astype(float)

    plt.subplots(dpi = 300)
    plt.imshow(df_HM, cmap=cm.hot, vmin=vmin_, vmax=vmax_*0.8, zorder=1) #cmap_w2a
    plt.xticks([])
    plt.yticks([])
    plt.box(False)
    plt.autoscale(enable=True, tight=True)

    plt.rcParams['figure.figsize'] = [2, 2]

    plt.savefig(f"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/MSImage/{name}_{mz}.tif", bbox_inches = 'tight', pad_inches = 0)

In [100]:
df_raMSI_ML = pd.read_csv(r'H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/raMSI_inDirectIn_noLT.csv').set_index('pixel_id')
df_raMSI_ML.shape

(377537, 4019)

In [101]:
raMSI_ML = df_raMSI_ML.reset_index()

In [102]:
raMSI_ML["Sample"] = 0
raMSI_ML["Row"] = 0
raMSI_ML["Scan"] = 0
Sample = []
Row = []
Scan = []
for i in range(len(raMSI_ML["pixel_id"])):
    Sample.append("_".join(str(raMSI_ML["pixel_id"][i]).split("_")[:-2]))
    Row.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-1]))
    Scan.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-2]))
raMSI_ML["Sample"] = Sample
raMSI_ML["Row"] = Row
raMSI_ML["Scan"] = Scan

In [103]:
pop_column = raMSI_ML.pop('Scan')
raMSI_ML.insert(1, 'Scan', pop_column)
pop_column = raMSI_ML.pop('Row')
raMSI_ML.insert(1, 'Row', pop_column)
pop_column = raMSI_ML.pop('Sample')
raMSI_ML.insert(1, 'Sample', pop_column)

In [104]:
raMSI_ML = raMSI_ML.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [105]:
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])


In [106]:
sample_list = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4, 
               HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3, 
               HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6, 
               HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01, 
               HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04, 
               HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05, 
               HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10, 
               HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12, 
               HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01, 
               HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04, 
               HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10, 
               HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11, 
               HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12]

In [108]:
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

# = 0
#TOP = "885.5488"

counter = 0
for TOP in TOP_features_XGB:
    for df in sample_ROI_list[:-10]:
        start = datetime.now()

        name = name_list[counter]
        sample_ = sample_list[counter]
        preview = sample_[[TOP]]
        for i in preview.columns:
            preview[i] = preview[i].astype(float)
        #vmin_list = list(preview.describe().iloc[-3])
        vmax_list = list(preview.describe().iloc[-1])
        vmin_ = 0
        vmax_ = vmax_list[0]
    #print(TOP, name, vmax_)

        genAROItif(sample_ROI=df, sample_=sample_, name=name, mz=TOP, vmin_=vmin_, vmax_=vmax_)
    #print(df.shape)
        counter += 1
        end = datetime.now()
        Duration = end - start
        print(Duration)
    counter = 0

0:00:47.638843
0:00:59.294942
0:01:09.183454
0:00:37.262875
0:00:43.393052
0:01:01.454418
0:00:03.872669
0:00:02.756367
0:00:04.221656
0:00:04.106214
0:02:14.360988
0:02:07.920842
0:02:17.604538
0:01:00.556651
0:00:50.677842
0:00:42.588835
0:00:42.260362
0:00:41.093813
0:00:41.856764
0:00:02.818232
0:00:06.064049
0:00:03.311330
0:00:02.931035
0:02:01.714834
0:02:04.029313
0:02:18.314359
0:00:51.455572
0:00:57.937821
0:01:07.231368
0:00:36.170771
0:00:42.196397
0:00:58.362527
0:00:03.877262
0:00:02.527646
0:00:03.992281
0:00:03.844888
0:02:10.506293
0:02:04.998427
0:02:15.172091
0:01:00.030259
0:00:50.546191
0:00:42.713435
0:00:42.276668
0:00:41.141718
0:00:41.975733
0:00:02.823118
0:00:06.193839
0:00:03.590982
0:00:02.895322
0:02:01.537151
0:02:03.331875
0:02:18.666592
0:00:51.882536
0:00:58.381277
0:01:07.752444
0:00:36.585417
0:00:43.263869
0:00:59.111114
0:00:03.878056
0:00:02.608407
0:00:04.074769
0:00:03.863043
0:02:10.366887
0:02:05.439826
0:02:15.670663
0:01:00.780606
0:00:51.18